In [1]:
import random
from math import exp, sqrt
def paths(S0, K, T, r, sigma, M, I):
        dt = T / M # 
        S = []
        for i in range(I):
            path = []
            for t in range(M + 1):
                if t == 0:
                    path.append(S0)
                else:
                    z = random.gauss(0.0, 1.0)
                    St = path[t - 1] * exp((r - 0.5 * sigma**2) * dt + sigma * sqrt(dt) * z)
                    path.append(St)
            S.append(path)

        return S

In [13]:
import numpy as np
from math import exp, sqrt
from scipy.linalg import lstsq
import time

def paths(S0, K, T, r, sigma, M, I):
    dt = T / M
    S = np.zeros((I, M + 1))
    S[:, 0] = S0
    for t in range(1, M + 1):
        z = np.random.standard_normal(I)
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * sqrt(dt) * z)
    return S

def payoff(S, K):
    return np.maximum(K - S, 0)

def discount_factor(r, t1, t0):
    return exp(-r * (t1 - t0))

def longstaff_schwartz(S, K, T, r, sigma, n_steps, n_paths, n_order, basis_functions):
    dt = T / n_steps
    V = np.zeros_like(S)

    # Step 2: Set prices at maturity to the payoff
    V[:, -1] = payoff(S[:, -1], K)

    # Steps 3-8: Iterate backwards in time
    for i in range(n_steps - 1, 0, -1):
        discount = discount_factor(r, (i + 1) * dt, i * dt)
        X = np.array([[basis_func(S[j, i]) for basis_func in basis_functions] for j in range(n_paths)])
        Y = discount * V[:, i + 1]

        # Step 6a: Solve regression coefficients
        beta, _, _, _ = lstsq(X, Y)

        # Step 6b: Compute continuation values
        continuation_values = X @ beta

        # Step 7: Exercise if immediate payoff is greater than continuation value
        immediate_payoff = payoff(S[:, i], K)
        V[:, i] = np.where(immediate_payoff > continuation_values, immediate_payoff, discount * V[:, i + 1])

    # Step 9-10: Calculate price today
    discounted_V = discount_factor(r, dt, 0) * V[:, 1]
    price_today = np.mean(discounted_V)
    standard_error = np.std(discounted_V) / sqrt(n_paths)
    
    return price_today, standard_error

# Define parameters
K = 100  # Strike price
r = 0.04  # Risk-free interest rate
T_values = [1, 2]  # Time to expiration
S0_values = [80, 85, 90, 95, 100, 105, 110, 115, 120]  # Initial stock prices
sigma_values = [0.2, 0.4]  # Volatility values
n_paths = 10000  # Number of paths
n_steps = 50  # Number of exercise points per year
n_order = 5  # Number of basis functions
basis_functions = [lambda x: 1, lambda x: x, lambda x: x**2, lambda x: x**3, lambda x: x**4]  # Example basis functions

# Print table header
print ("S(0)  | σ  | T   | Simulated American (MC)  | Standard Error | Time (s)")

# Loop through parameter combinations
for S0 in S0_values:
    for sigma in sigma_values:
        for T in T_values:
            start_time = time.time()
            S = paths(S0, K, T, r, sigma, n_steps, n_paths)
            price, std_err = longstaff_schwartz(S, K, T, r, sigma, n_steps, n_paths, n_order, basis_functions)
            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"{S0:<5} | {sigma:<3} | {T:<3} | {price:>12.4f} | {std_err:>14.4f} | {elapsed_time:>8.4f}")


S(0)  | σ  | T   | Simulated American (MC)  | Standard Error | Time (s)
80    | 0.2 | 1   |      19.9336 |         0.0461 |   4.4666
80    | 0.2 | 2   |      20.1815 |         0.0676 |   3.4369
80    | 0.4 | 1   |      24.4006 |         0.1633 |   3.1607
80    | 0.4 | 2   |      27.5485 |         0.1885 |   3.3025
85    | 0.2 | 1   |      15.5877 |         0.0762 |   3.2545
85    | 0.2 | 2   |      16.3104 |         0.0981 |   3.1213
85    | 0.4 | 1   |      21.1707 |         0.1582 |   3.1230
85    | 0.4 | 2   |      24.6730 |         0.1796 |   3.0065
90    | 0.2 | 1   |      11.8222 |         0.0842 |   3.1070
90    | 0.2 | 2   |      12.8923 |         0.1009 |   3.1491
90    | 0.4 | 1   |      18.0248 |         0.1467 |   3.0740
90    | 0.4 | 2   |      22.0631 |         0.1820 |   3.1861
95    | 0.2 | 1   |       8.4966 |         0.0793 |   3.0597
95    | 0.2 | 2   |      10.2352 |         0.0968 |   3.2336
95    | 0.4 | 1   |      15.8415 |         0.1449 |   3.1286
95    | 0.4 |

In [3]:
import random
from scipy.stats import norm
import numpy as np
from math import exp, sqrt

def paths(S0, K, T, r, sigma, M, I):
    dt = T / M
    S = []
    for i in range(I):
        path = []
        for t in range(M + 1):
            if t == 0:
                path.append(S0)
            else:
                z = random.gauss(0.0, 1.0)
                St = path[t - 1] * exp((r - 0.5 * sigma**2) * dt + sigma * sqrt(dt) * z)
                path.append(St)
        S.append(path)
    return S

def payoff(S, K):
    return max(K - S, 0)

def discount_factor(r, t1, t0):
    return exp(-r * (t1 - t0))

def longstaff_schwartz(S, K, T, r, sigma, n_steps, n_paths, n_order, basis_functions):
    dt = T / n_steps
    V = [[0] * (n_steps + 1) for _ in range(n_paths)]

    # Step 2: Set prices at maturity to the payoff
    for j in range(n_paths):
        V[j][-1] = payoff(S[j][-1], K)

    # Steps 3-8: Iterate backwards in time
    for i in range(n_steps - 1, 0, -1):
        discount = discount_factor(r, (i + 1) * dt, i * dt)
        X = [[basis_func(S[j][i]) for basis_func in basis_functions] for j in range(n_paths)]
        Y = [discount * V[j][i + 1] for j in range(n_paths)]

        # Step 6a: Solve regression coefficients
        beta = np.linalg.lstsq(X, Y, rcond=None)[0]

        # Step 6b: Compute continuation values
        continuation_values = [sum(beta[k] * basis_functions[k](S[j][i]) for k in range(n_order)) for j in range(n_paths)]

        # Step 7: Exercise if immediate payoff is greater than continuation value
        for j in range(n_paths):
            immediate_payoff = payoff(S[j][i], K)
            if immediate_payoff > continuation_values[j]:
                V[j][i] = immediate_payoff
            else:
                V[j][i] = discount * V[j][i + 1]

    # Step 9-10: Calculate price today
    price_today = sum(discount_factor(r, dt, 0) * V[j][1] for j in range(n_paths)) / n_paths
    return price_today

# Define parameters
K = 100  # Strike price
r = 0.04  # Risk-free interest rate
T_values = [1, 2]  # Time to expiration
S0_values = [80, 85, 90, 95, 100, 105, 110, 115, 120]  # Initial stock prices
sigma_values = [0.2, 0.4]  # Volatility values
n_paths = 10000  # Number of paths
n_steps = 50  # Number of exercise points per year
n_order = 5  # Number of basis functions
basis_functions = [lambda x: 1, lambda x: x, lambda x: x**2, lambda x: x**3, lambda x: x**4]  # Example basis functions

# Print table header
print ("S(0)  | σ  | T   | Simulated American (MC)  ")

# Loop through parameter combinations
for S0 in S0_values:
    for sigma in sigma_values:
        for T in T_values:
            S = paths(S0, K, T, r, sigma, n_steps, n_paths)
            price = longstaff_schwartz(S, K, T, r, sigma, n_steps, n_paths, n_order, basis_functions)
            
            print(f"{S0}  | {sigma} |  {T}    |  {price:12.4f}    ") 


S(0)  | σ  | T   | Simulated American (MC)  
80  | 0.2 |  1    |       19.9367    
80  | 0.2 |  2    |       20.2322    
80  | 0.4 |  1    |       24.0144    
80  | 0.4 |  2    |       27.3245    
85  | 0.2 |  1    |       15.4518    
85  | 0.2 |  2    |       16.2599    
85  | 0.4 |  1    |       20.8922    
85  | 0.4 |  2    |       24.8197    
90  | 0.2 |  1    |       11.6510    
90  | 0.2 |  2    |       12.9383    
90  | 0.4 |  1    |       18.3616    
90  | 0.4 |  2    |       22.3667    
95  | 0.2 |  1    |        8.7275    
95  | 0.2 |  2    |       10.3546    
95  | 0.4 |  1    |       16.0397    
95  | 0.4 |  2    |       19.9297    
100  | 0.2 |  1    |        6.2712    
100  | 0.2 |  2    |        8.1136    
100  | 0.4 |  1    |       13.7031    
100  | 0.4 |  2    |       18.4631    
105  | 0.2 |  1    |        4.4517    
105  | 0.2 |  2    |        6.4463    
105  | 0.4 |  1    |       11.8824    
105  | 0.4 |  2    |       16.6292    
110  | 0.2 |  1    |        3.1666 

In [8]:
import numpy as np
from math import exp, sqrt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings("ignore")
def paths(S0, K, T, r, sigma, M, I):
    dt = T / M
    S = np.zeros((I, M + 1))
    S[:, 0] = S0
    for t in range(1, M + 1):
        z = np.random.standard_normal(I)
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * sqrt(dt) * z)
    return S

def payoff(S, K):
    return np.maximum(K - S, 0)

def discount_factor(r, t1, t0):
    return exp(-r * (t1 - t0))

def longstaff_schwartz(S, K, T, r, sigma, n_steps, n_paths, model):
    dt = T / n_steps
    V = np.zeros_like(S)
    V[:, -1] = payoff(S[:, -1], K)
    
    for i in range(n_steps - 1, 0, -1):
        discount = discount_factor(r, (i + 1) * dt, i * dt)
        X = S[:, i].reshape(-1, 1)
        Y = discount * V[:, i + 1]

        model.fit(X, Y)
        continuation_values = model.predict(X)
        
        immediate_payoff = payoff(S[:, i], K)
        V[:, i] = np.where(immediate_payoff > continuation_values, immediate_payoff, discount * V[:, i + 1])
    
    discounted_V = discount_factor(r, dt, 0) * V[:, 1]
    price_today = np.mean(discounted_V)
    standard_error = np.std(discounted_V) / sqrt(n_paths)
    
    return price_today, standard_error

# Define parameters
K = 100  # Strike price
r = 0.04  # Risk-free interest rate
T_values = [1, 2]  # Time to expiration
S0_values = [80, 85, 90, 95, 100, 105, 110, 115, 120]  # Initial stock prices
sigma_values = [0.2, 0.4]  # Volatility values
n_paths = 10000  # Number of paths
n_steps = 50  # Number of exercise points per year

models = {
    'KNN': KNeighborsRegressor(n_neighbors=5),
    'Decision Tree': DecisionTreeRegressor(),
    'XGBoost': XGBRegressor(),
    'LightGBM': LGBMRegressor(),
    'Random Forest': RandomForestRegressor(n_estimators=100)
}

# Print table header
print("       | S(0)  | σ   | T   | KNN Price   | KNN StdErr   | DT Price    | DT StdErr    | XGB Price   | XGB StdErr    | RF Price    | RF StdErr   ")

# Loop through parameter combinations
for S0 in S0_values:
    for sigma in sigma_values:
        for T in T_values:
            prices_std_errs = []
            for model_name, model in models.items():
                S = paths(S0, K, T, r, sigma, n_steps, n_paths)
                price, std_err = longstaff_schwartz(S, K, T, r, sigma, n_steps, n_paths, model)
                prices_std_errs.extend([price, std_err])
            print(f"{S0:<7} | {sigma:<3} | {T:<3} | "
                  f"{prices_std_errs[0]:>10.4f} | {prices_std_errs[1]:>10.4f} | "
                  f"{prices_std_errs[2]:>10.4f} | {prices_std_errs[3]:>10.4f} | "
                  f"{prices_std_errs[4]:>10.4f} | {prices_std_errs[5]:>10.4f} | "
                  f"{prices_std_errs[6]:>10.4f} | {prices_std_errs[7]:>10.4f} | "
                  )


       | S(0)  | σ   | T   | KNN Price   | KNN StdErr   | DT Price    | DT StdErr    | XGB Price   | XGB StdErr    | RF Price    | RF StdErr   
80      | 0.2 | 1   |    24.3728 |     0.0748 |    28.8665 |     0.0744 |    21.0675 |     0.0599 |    28.0579 |     0.0787 | 
80      | 0.2 | 2   |    25.3591 |     0.0964 |    30.9987 |     0.0931 |    21.5372 |     0.0785 |    30.0077 |     0.0975 | 
80      | 0.4 | 1   |    30.8710 |     0.1459 |    38.3669 |     0.1333 |    26.1217 |     0.1488 |    37.1172 |     0.1404 | 
80      | 0.4 | 2   |    34.6132 |     0.1770 |    42.8716 |     0.1580 |    29.0901 |     0.1802 |    41.6898 |     0.1723 | 
85      | 0.2 | 1   |    19.8393 |     0.0818 |    24.4773 |     0.0800 |    16.5987 |     0.0674 |    23.6552 |     0.0829 | 
85      | 0.2 | 2   |    21.0375 |     0.1013 |    26.8311 |     0.1000 |    17.4713 |     0.0893 |    25.9148 |     0.1053 | 
85      | 0.4 | 1   |    27.0881 |     0.1518 |    34.6294 |     0.1438 |    22.7602 |     0.1

In [17]:
import numpy as np
from math import exp, sqrt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression

def paths(S0, K, T, r, sigma, M, I):
    dt = T / M
    S = np.zeros((I, M + 1))
    S[:, 0] = S0
    for t in range(1, M + 1):
        z = np.random.standard_normal(I)
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * sqrt(dt) * z)
    return S

def payoff(S, K):
    return np.maximum(K - S, 0)

def discount_factor(r, t1, t0):
    return exp(-r * (t1 - t0))

def longstaff_schwartz(S, K, T, r, sigma, n_steps, n_paths, threshold, n_order):
    dt = T / n_steps
    V = np.zeros_like(S)
    V[:, -1] = payoff(S[:, -1], K)
    
    for i in range(n_steps - 1, 0, -1):
        discount = discount_factor(r, (i + 1) * dt, i * dt)
        X = S[:, i].reshape(-1, 1)
        Y = discount * V[:, i + 1]

        # Fit Logistic Regression model with polynomial features
        poly = PolynomialFeatures(degree=n_order - 1)
        X_poly = poly.fit_transform(X)
        logreg_model = LogisticRegression()
        logreg_model.fit(X_poly, (Y > 0).astype(int))
        continuation_values_normalized = logreg_model.predict_proba(X_poly)[:, 1]
        continuation_values = expit(continuation_values_normalized)

        immediate_payoff = payoff(S[:, i], K)
        V[:, i] = np.where(immediate_payoff > continuation_values + threshold, immediate_payoff, discount * V[:, i + 1])
    
    discounted_V = discount_factor(r, dt, 0) * V[:, 1]
    price_today = np.mean(discounted_V)
    standard_error = np.std(discounted_V) / sqrt(n_paths)
    
    return price_today, standard_error

# Define parameters
K = 100  # Strike price
r = 0.04  # Risk-free interest rate
T_values = [1, 2]  # Time to expiration
S0_values = [80, 85, 90, 95, 100, 105, 110, 115, 120]  # Initial stock prices
sigma_values = [0.2, 0.4]  # Volatility values
n_paths = 10000  # Number of paths
n_steps = 50  # Number of exercise points per year
threshold = 10  # Threshold for exercising option
n_order = 3  # Polynomial degree for logistic regression

# Print table header
print("       | S(0)  | σ   | T   | Option Price | StdErr")

# Loop through parameter combinations
for S0 in S0_values:
    for sigma in sigma_values:
        for T in T_values:
            price, std_err = longstaff_schwartz(paths(S0, K, T, r, sigma, n_steps, n_paths), K, T, r, sigma, n_steps, n_paths, threshold, n_order)
            print(f"{S0:<7} | {sigma:<3} | {T:<3} | {price:>12.4f} | {std_err:>6.4f}")


       | S(0)  | σ   | T   | Option Price | StdErr
80      | 0.2 | 1   |      19.9017 | 0.0223
80      | 0.2 | 2   |      19.8808 | 0.0317
80      | 0.4 | 1   |      20.0072 | 0.0439
80      | 0.4 | 2   |      20.2990 | 0.0573
85      | 0.2 | 1   |      14.9996 | 0.0244
85      | 0.2 | 2   |      15.0473 | 0.0340
85      | 0.4 | 1   |      15.7338 | 0.0432
85      | 0.4 | 2   |      16.6420 | 0.0538
90      | 0.2 | 1   |      10.9473 | 0.0396
90      | 0.2 | 2   |      11.5759 | 0.0432
90      | 0.4 | 1   |      12.8650 | 0.0457
90      | 0.4 | 2   |      14.3169 | 0.0523
95      | 0.2 | 1   |       8.2624 | 0.0553
95      | 0.2 | 2   |       9.2995 | 0.0557
95      | 0.4 | 1   |      11.2985 | 0.0540
95      | 0.4 | 2   |      12.7907 | 0.0574
100     | 0.2 | 1   |       6.1403 | 0.0588
100     | 0.2 | 2   |       7.4502 | 0.0614
100     | 0.4 | 1   |       9.9796 | 0.0618
100     | 0.4 | 2   |      11.7455 | 0.0643
105     | 0.2 | 1   |       4.4016 | 0.0562
105     | 0.2 | 2   |    

In [ ]:
import numpy as np
from math import exp, sqrt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings("ignore")

def paths(S0, K, T, r, sigma, M, I):
    dt = T / M
    S = np.zeros((I, M + 1))
    S[:, 0] = S0
    for t in range(1, M + 1):
        z = np.random.standard_normal(I)
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * sqrt(dt) * z)
    return S

def payoff(S, K):
    return np.maximum(K - S, 0)

def discount_factor(r, t1, t0):
    return exp(-r * (t1 - t0))

def longstaff_schwartz(S, K, T, r, sigma, n_steps, n_paths, model):
    dt = T / n_steps
    V = np.zeros_like(S)
    V[:, -1] = payoff(S[:, -1], K)
    
    for i in range(n_steps - 1, 0, -1):
        discount = discount_factor(r, (i + 1) * dt, i * dt)
        X = S[:, i].reshape(-1, 1)
        Y = discount * V[:, i + 1]

        model.fit(X, Y)
        continuation_values = model.predict(X)
        
        immediate_payoff = payoff(S[:, i], K)
        V[:, i] = np.where(immediate_payoff > continuation_values, immediate_payoff, discount * V[:, i + 1])
    
    discounted_V = discount_factor(r, dt, 0) * V[:, 1]
    price_today = np.mean(discounted_V)
    standard_error = np.std(discounted_V) / sqrt(n_paths)
    
    return price_today, standard_error

# Define parameters
K = 100  # Strike price
r = 0.04  # Risk-free interest rate
T_values = [1, 2]  # Time to expiration
S0_values = [80, 85, 90, 95, 100, 105, 110, 115, 120]  # Initial stock prices
sigma_values = [0.2, 0.4]  # Volatility values
n_paths = 10000  # Number of paths
n_steps = 50  # Number of exercise points per year

models = {
    'KNN': KNeighborsRegressor(n_neighbors=5),
    'Decision Tree': DecisionTreeRegressor(),
    'XGBoost': XGBRegressor(),
    'LightGBM': LGBMRegressor(),
    'Random Forest': RandomForestRegressor(n_estimators=100)
}

# Print table header
print(f"{'S(0)':>7} | {'σ':>3} | {'T':>3} | {'KNN Price':>10} | {'KNN StdErr':>12} | "
      f"{'DT Price':>10} | {'DT StdErr':>12} | {'XGB Price':>10} | {'XGB StdErr':>12} | "
      f"{'RF Price':>10} | {'RF StdErr':>12}")

# Loop through parameter combinations
for S0 in S0_values:
    for sigma in sigma_values:
        for T in T_values:
            prices_std_errs = []
            for model_name, model in models.items():
                S = paths(S0, K, T, r, sigma, n_steps, n_paths)
                price, std_err = longstaff_schwartz(S, K, T, r, sigma, n_steps, n_paths, model)
                prices_std_errs.extend([price, std_err])
            print(f"{S0:>7} | {sigma:>3} | {T:>3} | "
                  f"{prices_std_errs[0]:>10.4f} | {prices_std_errs[1]:>12.4f} | "
                  f"{prices_std_errs[2]:>10.4f} | {prices_std_errs[3]:>12.4f} | "
                  f"{prices_std_errs[4]:>10.4f} | {prices_std_errs[5]:>12.4f} | "
                  f"{prices_std_errs[6]:>10.4f} | {prices_std_errs[7]:>12.4f} | "
                  f"{prices_std_errs[8]:>10.4f} | {prices_std_errs[9]:>12.4f}")
